# Pré-processamento de Textos em NLP

Este notebook realiza o pré-processamento de dois textos distintos:
1. **Matéria jornalística** do jornal A Tribuna
2. **Bula do medicamento** Doril

## Etapas do Pré-processamento:
1. Remover tags HTML
2. Remover URLs
3. Remover emojis
4. Remover stopwords
5. Remover sinais de pontuação
6. Remover caracteres especiais
7. Remover espaços em branco excedentes
8. Substituir palavras de chat por formas normais
9. Converter números em palavras
10. Converter para letras minúsculas
11. Aplicar correção ortográfica
12. Aplicar stemização
13. Aplicar lematização

In [17]:
# Instalação das bibliotecas necessárias
!pip install nltk spacy emoji pyspellchecker num2words beautifulsoup4
!python -m spacy download pt_core_news_sm

     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     --- ------------------------------------ 1.0/13.0 MB 5.0 MB/s eta 0:00:03
     -------- ------------------------------- 2.6/13.0 MB 6.0 MB/s eta 0:00:02
     ---------- ----------------------------- 3.4/13.0 MB 5.0 MB/s eta 0:00:02
     ------------- -------------------------- 4.5/13.0 MB 5.1 MB/s eta 0:00:02
     ---------------- ----------------------- 5.5/13.0 MB 5.2 MB/s eta 0:00:02
     -------------------- ------------------- 6.6/13.0 MB 5.2 MB/s eta 0:00:02
     ----------------------- ---------------- 7.6/13.0 MB 5.2 MB/s eta 0:00:02
     --------------------------- ------------ 8.9/13.0 MB 5.1 MB/s eta 0:00:01
     ------------------------------ --------- 10.0/13.0 MB 5.2 MB/s eta 0:00:01
     --------------------------------- ------ 11.0/13.0 MB 5.1 MB/s eta 0:00:01
     ------------------------------------ --- 11.8/13.0 MB 5.0 MB/s eta 0:00:01
     ---------------------------------------- 13.0/13.0 

In [18]:
# Importação das bibliotecas
import re
import string
import nltk
import spacy
import emoji
from spellchecker import SpellChecker
from num2words import num2words
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

# Downloads necessários do NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('rslp')

# Carregando modelo do spaCy para português
nlp = spacy.load('pt_core_news_sm')

print("Bibliotecas carregadas com sucesso!")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\helio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\helio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\helio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\helio\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


Bibliotecas carregadas com sucesso!


In [19]:
# Carregamento dos textos originais
with open('doril.txt', 'r', encoding='utf-8') as f:
    texto_doril = f.read()

with open('no14011801.txt', 'r', encoding='utf-8') as f:
    texto_jornal = f.read()

print("=== TEXTO ORIGINAL - BULA DORIL ===")
print(f"Tamanho: {len(texto_doril)} caracteres")
print(texto_doril[:500] + "...\n")

print("=== TEXTO ORIGINAL - JORNAL A TRIBUNA ===")
print(f"Tamanho: {len(texto_jornal)} caracteres")
print(texto_jornal[:500] + "...")

=== TEXTO ORIGINAL - BULA DORIL ===
Tamanho: 13742 caracteres
DORIL®
(ácido acetilsalicílico + cafeína)

Cosmed Indústria de Cosméticos e Medicamentos S.A.

Comprimido

500mg + 30mg

I - IDENTIFICAÇÃO DO MEDICAMENTO:
Doril®
ácido acetilsalicílico + cafeína
APRESENTAÇÕES
Comprimido.
Embalagens contendo 20 ou 150 comprimidos.
VIA DE ADMINISTRAÇÃO: ORAL
USO ADULTO
COMPOSIÇÃO
Cada comprimido contém:
ácido acetilsalicílico …………………………………………………….................................... 500mg
cafeína ………………………………….................……………………………….…...…............ 30mg
ex...

=== TEXTO ORIGINAL - JORNAL A TRIBUNA ===
Tamanho: 2634 caracteres
VITÓRIA-ES | DOMINGO, 14 DE JANEIRO DE 2018 | ANO LXXIX | Nº 26.236 | FUNDADO EM 22/09/1938 | EDIÇÃO DE 72 PÁGINAS Vo c ê conhece bem seus amigos? >AT 2 THIAGO COUTINHO/AT THIAGO COUTINHO/AT R$ 3,50 DEMAIS CIDADES R$ 3 ,0 0 GRANDE VITÓRIA AS S I N E 3323 -6333 Transexuais querem fazer parte da cota feminina nas próximas eleições >32 R efúgios ro m â n t i c o s no

## Definição das Funções de Pré-processamento

Cada função implementa uma etapa específica do pré-processamento.

In [20]:
# Etapa 1: Remover tags HTML
def step_1_remove_html_tags(text):
    """Remove tags HTML do texto"""
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

# Etapa 2: Remover URLs
def step_2_remove_urls(text):
    """Remove URLs do texto"""
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = url_pattern.sub('', text)
    # Remove também URLs simplificadas como t.co/...
    text = re.sub(r'\S*\.co/\S*', '', text)
    text = re.sub(r'www\.\S+', '', text)
    return text

# Etapa 3: Remover emojis
def step_3_remove_emojis(text):
    """Remove emojis do texto"""
    return emoji.replace_emoji(text, replace='')

# Etapa 4: Remover stopwords
def step_4_remove_stopwords(text):
    """Remove stopwords em português"""
    stop_words = set(stopwords.words('portuguese'))
    # Adicionar algumas stopwords customizadas
    stop_words.update(['que', 'de', 'a', 'o', 'e', 'do', 'da', 'em', 'um', 'para', 'é', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'à', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 'há', 'nos', 'já', 'está', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'suas', 'numa', 'nem', 'suas', 'meu', 'às', 'minha', 'têm', 'numa', 'pelos', 'pelas', 'essa', 'num', 'essa', 'foram', 'eles', 'estas', 'tinha', 'outro', 'essa', 'esses'])
    
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

print("Funções 1-4 definidas!")

Funções 1-4 definidas!


In [21]:
# Etapa 5: Remover sinais de pontuação
def step_5_remove_punctuation(text):
    """Remove sinais de pontuação"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# Etapa 6: Remover caracteres especiais (MELHORADA)
def step_6_remove_special_chars(text):
    """Remove caracteres especiais e normaliza repetições excessivas de letras"""
    # Primeiro, normaliza repetições excessivas de letras (mais de 2 iguais seguidas)
    # "PPPaaaiiisss" -> "PPaaiiss", "mmmééédddiiicccooosss" -> "mmééddiiccoos"
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    
    # Remove caracteres especiais, mantendo apenas letras, números e espaços
    text = re.sub(r'[^a-zA-ZÀ-ÿ0-9\s]', '', text)
    
    return text

# Etapa 7: Remover espaços excedentes
def step_7_remove_extra_spaces(text):
    """Remove espaços em branco excedentes"""
    return re.sub(r'\s+', ' ', text.strip())

# Etapa 8: Substituir palavras de chat (MELHORADA)
def step_8_normalize_chat_words(text):
    """Substitui abreviações e gírias por formas normais, incluindo palavras com repetições"""
    chat_dict = {
        'vc': 'você',
        'vcs': 'vocês',
        'tb': 'também',
        'tmbm': 'também',
        'pq': 'porque',
        'qq': 'qualquer',
        'qdo': 'quando',
        'ñ': 'não',
        'nao': 'não',
        'naoo': 'não',
        'naaoo': 'não',
        'kk': '',
        'kkk': '',
        'kkkk': '',
        'rs': '',
        'rsrs': '',
        'haha': '',
        'hehe': '',
        'blz': 'beleza',
        'flw': 'falou',
        'cmg': 'comigo',
        'ctg': 'contigo',
        'mt': 'muito',
        'mto': 'muito',
        'msm': 'mesmo',
        'td': 'tudo',
        'tds': 'todos',
        'bjo': 'beijo',
        'bjs': 'beijos',
        # Adiciona tratamento para palavras com repetições que sabemos que são comuns
        'sleepy': 'com sono',
        'sleeepy': 'com sono',
        'sleeeepy': 'com sono',
        'sleeeeepy': 'com sono'
    }
    
    words = text.split()
    normalized_words = []
    
    for word in words:
        word_lower = word.lower()
        
        # Verifica se a palavra está no dicionário diretamente
        if word_lower in chat_dict:
            replacement = chat_dict[word_lower]
            if replacement:  # Se não for string vazia
                normalized_words.append(replacement)
        else:
            # Tenta normalizar repetições antes de verificar o dicionário
            normalized_word = re.sub(r'(.)\1{2,}', r'\1', word_lower)
            if normalized_word in chat_dict:
                replacement = chat_dict[normalized_word]
                if replacement:  # Se não for string vazia
                    normalized_words.append(replacement)
            else:
                normalized_words.append(word)
    
    return ' '.join(normalized_words)

print("Funções 5-8 definidas e melhoradas!")

Funções 5-8 definidas e melhoradas!


In [22]:
# Etapa 9: Converter números em palavras (CORRIGIDA COMPLETAMENTE)
def step_9_convert_numbers_to_words(text):
    """Converte TODOS os números em palavras, incluindo números grandes e datas"""
    def replace_number(match):
        number = match.group()
        try:
            # Tenta converter para inteiro primeiro
            num = int(number)
            
            # Para números muito grandes (como datas), divide em partes menores
            if num > 100000:
                # Se parece com uma data, trata especialmente
                if len(number) == 8:
                    # Formato DDMMAAAA - divide em partes
                    day = int(number[:2])
                    month = int(number[2:4])
                    year = int(number[4:])
                    
                    day_words = num2words(day, lang='pt') if day <= 31 else num2words(day, lang='pt')
                    month_words = num2words(month, lang='pt') if month <= 12 else num2words(month, lang='pt')
                    year_words = num2words(year, lang='pt') if year <= 2100 else num2words(year, lang='pt')
                    
                    return f"{day_words} {month_words} {year_words}"
                else:
                    # Para outros números grandes, converte normalmente
                    return num2words(num, lang='pt')
            else:
                # Números normais - SEMPRE converte
                return num2words(num, lang='pt')
                
        except:
            try:
                # Tenta converter para float
                num = float(number)
                return num2words(int(num), lang='pt')  # Converte float para int primeiro
            except:
                return number
    
    # Encontra TODOS os números
    # Primeiro, converte números isolados
    text = re.sub(r'\b\d+(?:\.\d+)?\b', replace_number, text)
    
    # Remove números de nomes de usuário/tags mantendo o texto
    text = re.sub(r'(\w+)\d+', r'\1', text)
    
    return text

# Etapa 10: Converter para minúsculas
def step_10_to_lowercase(text):
    """Converte todo o texto para minúsculas"""
    return text.lower()

# Etapa 11: Correção ortográfica (MELHORADA)
def step_11_spell_correction(text):
    """Aplica correção ortográfica melhorada para tratar repetições"""
    spell = SpellChecker(language='pt')
    words = text.split()
    corrected_words = []
    
    for word in words:
        if len(word) > 2:  # Só corrige palavras com mais de 2 caracteres
            
            # Primeiro, normaliza repetições excessivas na palavra
            normalized_word = re.sub(r'(.)\1{2,}', r'\1', word.lower())
            
            # Verifica se a palavra normalizada está no dicionário
            if normalized_word in spell:
                corrected_words.append(normalized_word)
            else:
                # Tenta corrigir a palavra normalizada
                correction = spell.correction(normalized_word)
                if correction and correction != normalized_word:
                    corrected_words.append(correction)
                else:
                    # Se não conseguiu corrigir, tenta algumas normalizações comuns
                    # Remove todas as repetições deixando apenas uma letra
                    single_char_word = re.sub(r'(.)\1+', r'\1', word.lower())
                    if single_char_word in spell:
                        corrected_words.append(single_char_word)
                    else:
                        single_correction = spell.correction(single_char_word)
                        if single_correction and single_correction != single_char_word:
                            corrected_words.append(single_correction)
                        else:
                            # Se ainda não conseguiu, mantém a palavra original normalizada
                            corrected_words.append(normalized_word)
        else:
            corrected_words.append(word)
    
    return ' '.join(corrected_words)

print("Funções 9-11 definidas e melhoradas!")

Funções 9-11 definidas e melhoradas!


In [23]:
# Etapa 12: Stemização
def step_12_stemming(text):
    """Aplica stemização usando RSLP Stemmer"""
    stemmer = RSLPStemmer()
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

# Etapa 13: Lematização 
def step_13_lemmatization(text):
    """Aplica lematização usando spaCy, com tratamento especial para palavras stemizadas"""
    
    # Dicionário para corrigir palavras comuns que ficaram stemizadas
    stem_to_lemma = {
        # Números
        'vint': 'vinte', 'trint': 'trinta', 'quarent': 'quarenta', 'cinquent': 'cinquenta',
        'catorz': 'catorze', 'quinz': 'quinze', 'doiz': 'doze', 'trez': 'treze', 
        'dezoitr': 'dezoito', 'seil': 'seis', 'oit': 'oito', 'quatr': 'quatro',
        'cinc': 'cinco', 'nov': 'nove', 'set': 'sete', 'trê': 'três',
        
        # Termos médicos (específicos da bula)
        'medic': 'médico', 'comprim': 'comprimido', 'ácid': 'ácido', 'analgés': 'analgésico',
        'antipoé': 'antipirético', 'func': 'funcionar', 'contraindic': 'contraindicado',
        'paci': 'paciente', 'administr': 'administrar', 'oral': 'oral', 'adult': 'adulto',
        'compo': 'composição', 'cad': 'cada', 'contémr': 'contém', 'excipir': 'excipiente',
        'amid': 'amido', 'cor': 'cor', 'vermelh': 'vermelho', 'bul': 'bula',
        'inforr': 'informar', 'indic': 'indicado', 'espec': 'especial', 'trat': 'tratar',
        'reduç': 'redução', 'febr': 'febre', 'cabeç': 'cabeça', 'açã': 'ação',
        'antipiré': 'antipirético', 'portant': 'portanto', 'atu': 'atuar', 'alivi': 'aliviar',
        'possur': 'possuir', 'subst': 'substância', 'potencializ': 'potencializar',
        'apresent': 'apresentar', 'aind': 'ainda', 'efeit': 'efeito', 'estimul': 'estimular',
        'hum': 'humano', 'est': 'estar', 'alert': 'alerta', 'atenç': 'atenção',
        'promov': 'promover', 'constr': 'constrição', 'diminu': 'diminuir', 'calibr': 'calibre',
        'sanguíne': 'sanguíneo', 'cérebrr': 'cerebral', 'pod': 'poder', 'contribu': 'contribuir',
        'dev': 'dever', 'us': 'usar', 'hipersensibil': 'hipersensibilidade', 'outr': 'outro',
        'salicilat': 'salicilato', 'qualqu': 'qualquer', 'compon': 'componente', 'fórmul': 'fórmula',
        'produt': 'produto', 'predispostr': 'predisposto', 'dispeps': 'dispepsia', 'indigest': 'indigestão',
        'sab': 'saber', 'portr': 'portar', 'algum': 'algum', 'Les': 'lesão', 'muco': 'mucosa',
        'gástr': 'gástrica', 'úlc': 'úlcera', 'intoler': 'intolerância', 'hepá': 'hepática',
        'grav': 'grave', 'além': 'além', 'hemofíl': 'hemofilia', 'problem': 'problema',
        'sangrent': 'sangramento', 'tom': 'tomar', 'cuid': 'cuidado', 'renal': 'renal',
        'compromet': 'comprometida', 'Prim': 'primeiro', 'mes': 'mês', 'Gravid': 'gravidez',
        'após': 'após', 'períod': 'período', 'empreg': 'emprego', 'ca': 'caso', 'absolut': 'absoluto',
        
        # Profissões e substantivos
        'ministr': 'ministro', 'presid': 'presidente', 'pilot': 'piloto', 'líd': 'líder', 'chef': 'chefe',
        
        # Palavras comuns
        'famíl': 'família', 'empr': 'empresa', 'cidad': 'cidade', 'pesso': 'pessoa',
        'part': 'parte', 'grand': 'grande', 'feminin': 'feminino', 'próx': 'próximo',
        'refúgi': 'refúgio', 'amig': 'amigo', 'fund': 'fundar', 'ediç': 'edição', 
        'págin': 'página', 'comiss': 'comissão', 'públic': 'público', 'film': 'filme',
        'parc': 'parque', 'especi': 'especial', 'dilem': 'dilema', 'carg': 'cargo',
        'ritm': 'ritmo', 'internac': 'internacional', 'relatóri': 'relatório',
        'sobr': 'sobre', 'fest': 'festa', 'liberdadr': 'liberdade',
        
        # Verbos
        'conhec': 'conhecer', 'gost': 'gostar', 'ach': 'achar', 'trabalh': 'trabalhar',
        'represent': 'representar', 'imagin': 'imaginar', 'aprend': 'aprender', 
        'pergunt': 'perguntar', 'concorr': 'concorrer', 'compr': 'comprar',
        'ajud': 'ajudar', 'deix': 'deixar', 'conver': 'conversar',
        'pression': 'pressionar', 'investig': 'investigar', 'coloc': 'colocar',
        'desej': 'desejar', 'divulg': 'divulgar', 'pass': 'passar', 'cadastr': 'cadastrar',
        'apaixon': 'apaixonar', 'entr': 'entrar', 'entrr': 'entrar',
        
        # Adjetivos e outros
        'vici': 'viciado', 'depend': 'dependente', 'gov': 'governo', 'decis': 'decisão',
        'risc': 'risco', 'vergonh': 'vergonha', 'aul': 'aula', 'famili': 'família'
    }
    
    words = text.split()
    lemmatized_words = []
    
    for word in words:
        # Primeiro verifica se a palavra stemizada tem correção no dicionário
        if word.lower() in stem_to_lemma:
            lemmatized_words.append(stem_to_lemma[word.lower()])
        else:
            # Tenta aplicar lematização normal com spaCy
            try:
                doc = nlp(word)
                if len(doc) > 0 and doc[0].lemma_ != word and doc[0].lemma_ != '-PRON-':
                    lemmatized_words.append(doc[0].lemma_)
                else:
                    
                    best_word = word
                    for suffix in ['', 'a', 'o', 'e', 'ar', 'er', 'ir', 'ão']:
                        test_word = word + suffix
                        try:
                            test_doc = nlp(test_word)
                            if len(test_doc) > 0 and test_doc[0].is_alpha and not test_doc[0].is_oov:
                                lemma = test_doc[0].lemma_
                                if lemma != '-PRON-' and len(lemma) > 2:
                                    best_word = lemma
                                    break
                        except:
                            continue
                    lemmatized_words.append(best_word)
            except:
                lemmatized_words.append(word)
    
    return ' '.join(lemmatized_words)

print("Funções 12-13 definidas!")
print("\nTodas as funções de pré-processamento foram definidas com sucesso!")

Funções 12-13 definidas!

Todas as funções de pré-processamento foram definidas com sucesso!


In [24]:

print("=== TESTE DA LEMATIZAÇÃO MELHORADA ===")


texto_stemizado = "vint trint catorz médic famíl conhec gost fazer trabalh gov ministr represent"
print(f"Texto stemizado: {texto_stemizado}")


resultado_lemmatizado = step_13_lemmatization(texto_stemizado)
print(f"Após lematização: {resultado_lemmatizado}")

print("\n=== COMPARAÇÃO PALAVRA POR PALAVRA ===")
palavras_stem = texto_stemizado.split()
palavras_lemma = resultado_lemmatizado.split()

for i, (stem, lemma) in enumerate(zip(palavras_stem, palavras_lemma)):
    print(f"{i+1:2d}. {stem:10} -> {lemma}")

print("\n✅ Teste da lematização concluído!")

=== TESTE DA LEMATIZAÇÃO MELHORADA ===
Texto stemizado: vint trint catorz médic famíl conhec gost fazer trabalh gov ministr represent
Após lematização: vinte trinta catorze Médic família conhecer gostar fazer trabalhar governo ministro representar

=== COMPARAÇÃO PALAVRA POR PALAVRA ===
 1. vint       -> vinte
 2. trint      -> trinta
 3. catorz     -> catorze
 4. médic      -> Médic
 5. famíl      -> família
 6. conhec     -> conhecer
 7. gost       -> gostar
 8. fazer      -> fazer
 9. trabalh    -> trabalhar
10. gov        -> governo
11. ministr    -> ministro
12. represent  -> representar

✅ Teste da lematização concluído!


In [25]:
# Teste da conversão de números melhorada
print("=== TESTE DA CONVERSÃO DE NÚMEROS ===")

texto_com_numeros = "22091938 fundação em 1998 nickcristan17 500 reais"
print(f"Texto original: {texto_com_numeros}")

resultado_numeros = step_9_convert_numbers_to_words(texto_com_numeros)
print(f"Após conversão: {resultado_numeros}")

print("\n=== TESTE COM MAIS PALAVRAS STEMIZADAS ===")
texto_stem_completo = "seil oit quatr cinc nov set entrr liberdadr empr divulg"
print(f"Texto stemizado: {texto_stem_completo}")

resultado_lemma_completo = step_13_lemmatization(texto_stem_completo)
print(f"Após lematização: {resultado_lemma_completo}")

print("\n✅ Testes concluídos!")

=== TESTE DA CONVERSÃO DE NÚMEROS ===
Texto original: 22091938 fundação em 1998 nickcristan17 500 reais
Após conversão: vinte e dois nove mil novecentos e trinta e oito fundação em mil novecentos e noventa e oito nickcristan1 quinhentos reais

=== TESTE COM MAIS PALAVRAS STEMIZADAS ===
Texto stemizado: seil oit quatr cinc nov set entrr liberdadr empr divulg
Após lematização: seis oito quatro cinco nove sete entrar liberdade empresa divulgar

✅ Testes concluídos!


## Aplicação Sequencial do Pré-processamento

Agora vamos aplicar todas as etapas sequencialmente e mostrar o resultado após cada transformação.

In [26]:
# Teste das melhorias com o texto problemático
texto_teste = "PPPaaaiiisss pppeeedddeeemmm aaajjjuuudddaaa aaa mmmééédddiiicccooosss pppaaarrraaa tttrrraaatttaaarrr fffiiilllhhhooosss vvviiiccciiiaaadddooosss"

print("=== TESTE DAS MELHORIAS ===")
print(f"Texto original:\n{texto_teste}")

print(f"\nApós etapa 6 (remover caracteres especiais):")
teste_etapa6 = step_6_remove_special_chars(texto_teste)
print(teste_etapa6)

print(f"\nApós etapa 8 (normalizar chat):")
teste_etapa8 = step_8_normalize_chat_words(teste_etapa6)
print(teste_etapa8)

print(f"\nApós etapa 10 (minúsculas):")
teste_etapa10 = step_10_to_lowercase(teste_etapa8)
print(teste_etapa10)

print(f"\nApós etapa 11 (correção ortográfica):")
teste_etapa11 = step_11_spell_correction(teste_etapa10)
print(teste_etapa11)

print("\n" + "="*50)

=== TESTE DAS MELHORIAS ===
Texto original:
PPPaaaiiisss pppeeedddeeemmm aaajjjuuudddaaa aaa mmmééédddiiicccooosss pppaaarrraaa tttrrraaatttaaarrr fffiiilllhhhooosss vvviiiccciiiaaadddooosss

Após etapa 6 (remover caracteres especiais):
PPaaiiss ppeeddeemm aajjuuddaa aa mmééddiiccooss ppaarraa ttrraattaarr ffiillhhooss vviicciiaaddooss

Após etapa 8 (normalizar chat):
PPaaiiss ppeeddeemm aajjuuddaa aa mmééddiiccooss ppaarraa ttrraattaarr ffiillhhooss vviicciiaaddooss

Após etapa 10 (minúsculas):
ppaaiiss ppeeddeemm aajjuuddaa aa mmééddiiccooss ppaarraa ttrraattaarr ffiillhhooss vviicciiaaddooss

Após etapa 11 (correção ortográfica):
pais pedem ajuda aa médicos para tratar filhos viciados

pais pedem ajuda aa médicos para tratar filhos viciados



In [27]:
def apply_preprocessing_steps(text, text_name):
    """Aplica todas as etapas de pré-processamento sequencialmente"""
    
    steps = [
        ("Original", lambda x: x),
        ("1. Remover HTML", step_1_remove_html_tags),
        ("2. Remover URLs", step_2_remove_urls),
        ("3. Remover emojis", step_3_remove_emojis),
        ("4. Remover stopwords", step_4_remove_stopwords),
        ("5. Remover pontuação", step_5_remove_punctuation),
        ("6. Remover caracteres especiais", step_6_remove_special_chars),
        ("7. Remover espaços excedentes", step_7_remove_extra_spaces),
        ("8. Normalizar palavras de chat", step_8_normalize_chat_words),
        ("9. Converter números", step_9_convert_numbers_to_words),
        ("10. Converter para minúsculas", step_10_to_lowercase),
        ("11. Correção ortográfica", step_11_spell_correction),
        ("12. Stemização", step_12_stemming),
        ("13. Lematização", step_13_lemmatization)
    ]
    
    current_text = text
    results = {}
    
    print(f"\n{'='*60}")
    print(f"PRÉ-PROCESSAMENTO: {text_name.upper()}")
    print(f"{'='*60}")
    
    for step_name, step_function in steps:
        if step_name != "Original":
            current_text = step_function(current_text)
        
        results[step_name] = current_text
        
        print(f"\n{'-'*40}")
        print(f"ETAPA: {step_name}")
        print(f"{'-'*40}")
        print(f"Tamanho: {len(current_text)} caracteres")
        print(f"Palavras: {len(current_text.split())} palavras")
        
        # Mostra uma amostra do texto (primeiros 300 caracteres)
        sample = current_text[:300] + "..." if len(current_text) > 300 else current_text
        print(f"Amostra: {sample}")
    
    return results

print("Função de aplicação sequencial definida!")

Função de aplicação sequencial definida!


In [28]:
# Aplicando pré-processamento na BULA DO DORIL
resultados_doril = apply_preprocessing_steps(texto_doril, "Bula do Doril")


PRÉ-PROCESSAMENTO: BULA DO DORIL

----------------------------------------
ETAPA: Original
----------------------------------------
Tamanho: 13742 caracteres
Palavras: 1955 palavras
Amostra: DORIL®
(ácido acetilsalicílico + cafeína)

Cosmed Indústria de Cosméticos e Medicamentos S.A.

Comprimido

500mg + 30mg

I - IDENTIFICAÇÃO DO MEDICAMENTO:
Doril®
ácido acetilsalicílico + cafeína
APRESENTAÇÕES
Comprimido.
Embalagens contendo 20 ou 150 comprimidos.
VIA DE ADMINISTRAÇÃO: ORAL
USO ADUL...

----------------------------------------
ETAPA: 1. Remover HTML
----------------------------------------
Tamanho: 13742 caracteres
Palavras: 1955 palavras
Amostra: DORIL®
(ácido acetilsalicílico + cafeína)

Cosmed Indústria de Cosméticos e Medicamentos S.A.

Comprimido

500mg + 30mg

I - IDENTIFICAÇÃO DO MEDICAMENTO:
Doril®
ácido acetilsalicílico + cafeína
APRESENTAÇÕES
Comprimido.
Embalagens contendo 20 ou 150 comprimidos.
VIA DE ADMINISTRAÇÃO: ORAL
USO ADUL...

----------------------------------

In [29]:
# Aplicando pré-processamento no JORNAL A TRIBUNA
resultados_jornal = apply_preprocessing_steps(texto_jornal, "Jornal A Tribuna")


PRÉ-PROCESSAMENTO: JORNAL A TRIBUNA

----------------------------------------
ETAPA: Original
----------------------------------------
Tamanho: 2634 caracteres
Palavras: 413 palavras
Amostra: VITÓRIA-ES | DOMINGO, 14 DE JANEIRO DE 2018 | ANO LXXIX | Nº 26.236 | FUNDADO EM 22/09/1938 | EDIÇÃO DE 72 PÁGINAS Vo c ê conhece bem seus amigos? >AT 2 THIAGO COUTINHO/AT THIAGO COUTINHO/AT R$ 3,50 DEMAIS CIDADES R$ 3 ,0 0 GRANDE VITÓRIA AS S I N E 3323 -6333 Transexuais querem fazer parte da cota ...

----------------------------------------
ETAPA: 1. Remover HTML
----------------------------------------
Tamanho: 2634 caracteres
Palavras: 413 palavras
Amostra: VITÓRIA-ES | DOMINGO, 14 DE JANEIRO DE 2018 | ANO LXXIX | Nº 26.236 | FUNDADO EM 22/09/1938 | EDIÇÃO DE 72 PÁGINAS Vo c ê conhece bem seus amigos? >AT 2 THIAGO COUTINHO/AT THIAGO COUTINHO/AT R$ 3,50 DEMAIS CIDADES R$ 3 ,0 0 GRANDE VITÓRIA AS S I N E 3323 -6333 Transexuais querem fazer parte da cota ...

-----------------------------------

## Comparação Final: Original vs Processado

Vamos comparar o texto original com o texto final após todas as transformações.

In [30]:
def show_final_comparison(results, text_name):
    """Mostra comparação entre texto original e final"""
    
    original = results["Original"]
    final = results["13. Lematização"]
    
    print(f"\n{'='*80}")
    print(f"COMPARAÇÃO FINAL: {text_name.upper()}")
    print(f"{'='*80}")
    
    print(f"\n ESTATÍSTICAS:")
    print(f"   Original: {len(original)} caracteres, {len(original.split())} palavras")
    print(f"   Final: {len(final)} caracteres, {len(final.split())} palavras")
    print(f"   Redução: {((len(original) - len(final)) / len(original) * 100):.1f}% dos caracteres")
    
    print(f"\n TEXTO ORIGINAL (primeiros 500 caracteres):")
    print(f"   {original[:500]}...")
    
    print(f"\n🔧 TEXTO FINAL (primeiros 500 caracteres):")
    print(f"   {final[:500]}...")
    
    return original, final

# Comparações finais
original_doril, final_doril = show_final_comparison(resultados_doril, "Bula do Doril")
original_jornal, final_jornal = show_final_comparison(resultados_jornal, "Jornal A Tribuna")


COMPARAÇÃO FINAL: BULA DO DORIL

 ESTATÍSTICAS:
   Original: 13742 caracteres, 1955 palavras
   Final: 9136 caracteres, 1361 palavras
   Redução: 33.5% dos caracteres

 TEXTO ORIGINAL (primeiros 500 caracteres):
   DORIL®
(ácido acetilsalicílico + cafeína)

Cosmed Indústria de Cosméticos e Medicamentos S.A.

Comprimido

500mg + 30mg

I - IDENTIFICAÇÃO DO MEDICAMENTO:
Doril®
ácido acetilsalicílico + cafeína
APRESENTAÇÕES
Comprimido.
Embalagens contendo 20 ou 150 comprimidos.
VIA DE ADMINISTRAÇÃO: ORAL
USO ADULTO
COMPOSIÇÃO
Cada comprimido contém:
ácido acetilsalicílico …………………………………………………….................................... 500mg
cafeína ………………………………….................……………………………….…...…............ 30mg
ex...

🔧 TEXTO FINAL (primeiros 500 caracteres):
   doril ácido acetilsalicíl Cafeín cosm indústr Cosmé médico sa comprimido 50 um i identific médico doril ácido acetilsalicíl Cafeín apresentar comprimido embal cont vinte cent e cinquenta comprimido ver administrar oral uso adulto comp

## Salvando os Resultados

Vamos salvar os textos processados em arquivos para referência futura.

In [31]:
# Função para formatar texto de forma legível
def formatar_texto_legivel(texto, palavras_por_linha=10):
    """Formata o texto quebrando em linhas com número limitado de palavras"""
    palavras = texto.split()
    linhas = []
    linha_atual = []
    
    for palavra in palavras:
        linha_atual.append(palavra)
        if len(linha_atual) >= palavras_por_linha:
            linhas.append(' '.join(linha_atual))
            linha_atual = []
    
    # Adiciona a última linha se sobrou algo
    if linha_atual:
        linhas.append(' '.join(linha_atual))
    
    return '\n'.join(linhas)

# Salvando os resultados finais FORMATADOS EM LINHAS
print(" Salvando textos processados formatados...")

# Formata os textos
doril_formatado = formatar_texto_legivel(final_doril, palavras_por_linha=10)
jornal_formatado = formatar_texto_legivel(final_jornal, palavras_por_linha=10)

# Salva apenas as versões formatadas em linhas
with open('doril_processado.txt', 'w', encoding='utf-8') as f:
    f.write("=== BULA DORIL - TEXTO PROCESSADO (FORMATADO) ===\n\n")
    f.write(doril_formatado)

with open('jornal_processado.txt', 'w', encoding='utf-8') as f:
    f.write("=== JORNAL A TRIBUNA - TEXTO PROCESSADO (FORMATADO) ===\n\n")
    f.write(jornal_formatado)

# Salvando log detalhado de todas as etapas
with open('log_preprocessamento.txt', 'w', encoding='utf-8') as f:
    f.write("RELATÓRIO DE PRÉ-PROCESSAMENTO DE TEXTOS\n")
    f.write("=" * 60 + "\n\n")
    
    for texto_nome, resultados in [("BULA DORIL", resultados_doril), ("JORNAL A TRIBUNA", resultados_jornal)]:
        f.write(f"\n {texto_nome}\n")
        f.write("-" * 40 + "\n")
        
        for etapa, conteudo in resultados.items():
            f.write(f"\n {etapa}:\n")
            f.write(f"   Caracteres: {len(conteudo)}\n")
            f.write(f"   Palavras: {len(conteudo.split())}\n")
            
            # Formata uma amostra para o log
            if len(conteudo) > 200:
                amostra_formatada = formatar_texto_legivel(conteudo[:200], palavras_por_linha=8)
                f.write(f"   Amostra:\n")
                for linha in amostra_formatada.split('\n')[:3]:
                    f.write(f"      {linha}\n")
                f.write(f"      ...\n")
            else:
                amostra_formatada = formatar_texto_legivel(conteudo, palavras_por_linha=8)
                f.write(f"   Conteúdo:\n")
                for linha in amostra_formatada.split('\n'):
                    f.write(f"      {linha}\n")
            f.write("\n")

print(" Resultados salvos em:")
print("    TEXTOS FORMATADOS:")
print("      - doril_processado.txt")
print("      - jornal_processado.txt")
print("    LOG:")
print("      - log_preprocessamento.txt")
print("\n Pré-processamento concluído com textos formatados!")

 Salvando textos processados formatados...
 Resultados salvos em:
    TEXTOS FORMATADOS:
      - doril_processado.txt
      - jornal_processado.txt
    LOG:
      - log_preprocessamento.txt

 Pré-processamento concluído com textos formatados!


In [32]:
# Teste específico para números que ainda estão no Doril
print("=== TESTE DE CONVERSÃO DOS NÚMEROS PROBLEMÁTICOS ===")

# Simula os números que ainda estão presentes no arquivo
numeros_teste = "50 500 30 65 8 700 2 4 1 610824260020"
print(f"Números originais: {numeros_teste}")

# Aplica apenas a função de conversão de números
resultado_conversao = step_9_convert_numbers_to_words(numeros_teste)
print(f"Após conversão: {resultado_conversao}")

print("\n=== TESTE INDIVIDUAL ===")
for num in ["50", "500", "30", "65", "700"]:
    convertido = step_9_convert_numbers_to_words(num)
    print(f"{num} -> {convertido}")

print("\n✅ Teste de conversão concluído!")

=== TESTE DE CONVERSÃO DOS NÚMEROS PROBLEMÁTICOS ===
Números originais: 50 500 30 65 8 700 2 4 1 610824260020
Após conversão: cinquenta quinhentos trinta sessenta e cinco oito setecentos dois quatro um seiscentos e dez mil oitocentos e vinte e quatro milhões duzentos e sessenta mil vinte

=== TESTE INDIVIDUAL ===
50 -> cinquenta
500 -> quinhentos
30 -> trinta
65 -> sessenta e cinco
700 -> setecentos

✅ Teste de conversão concluído!


In [33]:
# Correção direta dos números no arquivo do Doril
print("=== CORRIGINDO NÚMEROS NO ARQUIVO DORIL_PROCESSADO.TXT ===")

# Lê o arquivo atual
with open('doril_processado.txt', 'r', encoding='utf-8') as f:
    conteudo_doril = f.read()

print(f"Arquivo carregado: {len(conteudo_doril)} caracteres")

# Identifica números presentes
import re
numeros_encontrados = re.findall(r'\b\d+\b', conteudo_doril)
print(f"Números encontrados: {set(numeros_encontrados)}")

# Remove o cabeçalho e aplica correção apenas no conteúdo
linhas = conteudo_doril.split('\n')
cabecalho = linhas[0] + '\n\n'  # Preserva cabeçalho
conteudo_apenas = '\n'.join(linhas[2:])  # Pega só o conteúdo

print("Aplicando correção de números...")
conteudo_corrigido = step_9_convert_numbers_to_words(conteudo_apenas)

# Verifica se a correção funcionou
numeros_apos_correcao = re.findall(r'\b\d+\b', conteudo_corrigido)
print(f"Números após correção: {set(numeros_apos_correcao) if numeros_apos_correcao else 'NENHUM! ✅'}")

# Salva o arquivo corrigido
arquivo_corrigido = cabecalho + conteudo_corrigido

with open('doril_processado.txt', 'w', encoding='utf-8') as f:
    f.write(arquivo_corrigido)

print(" Arquivo doril_processado.txt atualizado com números convertidos!")

# Mostra uma amostra do resultado
print("\n=== AMOSTRA DO RESULTADO ===")
print(arquivo_corrigido[:500] + "...")

=== CORRIGINDO NÚMEROS NO ARQUIVO DORIL_PROCESSADO.TXT ===
Arquivo carregado: 9187 caracteres
Números encontrados: {'50'}
Aplicando correção de números...
Números após correção: NENHUM! ✅
 Arquivo doril_processado.txt atualizado com números convertidos!

=== AMOSTRA DO RESULTADO ===
=== BULA DORIL - TEXTO PROCESSADO (FORMATADO) ===

doril ácido acetilsalicíl Cafeín cosm indústr Cosmé médico sa comprimido
cinquenta um i identific médico doril ácido acetilsalicíl Cafeín apresentar
comprimido embal cont vinte cent e cinquenta comprimido ver administrar
oral uso adulto compo cada comprimido contémr ácido acetilsalicíl cinquenta
Cafeín um excipir psp um comprimido amido cor vermelho eritrosin
n três doril comprimido bula paciente um ii inforr paciente
um médico indicado doril in...
